# 💬 Chat-Commerce 🛒

In [ ]:
# Install packages
%pip install python-dotenv langchain llama-index

In [19]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

True

In [20]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [21]:
from langchain.llms import OpenAIChat
from langchain import OpenAI

# Initialize OpenAI LLMs

# one for 'Chat'
llm_chat = OpenAIChat(temperature=0)

# one for 'Index'
llm_index = OpenAI(temperature=0)

In [22]:
from pathlib import Path
from llama_index import download_loader

# Read the data from CSV
SimpleCSVReader = download_loader("SimpleCSVReader")
loader = SimpleCSVReader()
documents = loader.load_data(file=Path('./data/home-and-garden.csv'))

In [ ]:
from llama_index import GPTSimpleVectorIndex, LLMPredictor

# Index the data and store the embeddings
# NOTE: Run this only if you want to re-index
llm_predictor = LLMPredictor(llm=llm_index)
index_data = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor)
index_data.save_to_disk("./index/index.json")

In [23]:
from llama_index import GPTSimpleVectorIndex

# Load the embeddings from existing index
index_data = GPTSimpleVectorIndex.load_from_disk('./index/index.json')

In [24]:
from langchain.agents import Tool

# Create a langchain 'Tool'
tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(index_data.query(q)),
        description="useful for when you want to answer questions about products available in the store. The store sells furnitures and items for home and garden",
        return_direct=True
    ),
]

In [25]:
from llama_index import GPTListIndex
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

# Index to be used for chat history
index_chat_history = GPTListIndex([])

# chat history
memory = GPTIndexChatMemory(
    index=index_chat_history, 
    memory_key="chat_history", 
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)

# create an agent
agent_chain = initialize_agent(tools, llm=llm_chat, agent="conversational-react-description", memory=memory)

In [26]:
agent_chain.run(input="hi, i am bob")

'Hello Bob! How can I assist you today?'

In [27]:
agent_chain.run(input="what are the products available?")

'\nThe products available in the store are: Clay Plant Pot, Copper Light, Cream Sofa, Antique Drawers, White Bed Clothes, Pink Armchair, Wooden Outdoor Table, Brown Throw Pillows, White Ceramic Pot, Yellow Watering Can, Gardening Hand Trowel, Biodegradable Cardboard Pots, Grey Sofa, Wooden Outdoor Slats, Wooden Fence, Yellow Sofa, Knitted Throw Pillows, Vanilla Candle, Black Bean Bag, Bedside Table.'

In [28]:
agent_chain.run(input="I'm looking for some furniture to use in my garden")

'\nYes, garden furniture is included in the context information. Examples of garden furniture include the Wooden Outdoor Table, Wooden Outdoor Slats, Wooden Fence, and Biodegradable Cardboard Pots.'

In [29]:
agent_chain.run(input="Great. Can you pick me one table?")

'\nVendor: Rustic LTD\nType: Outdoor\nTags: Wood, Garden\nPrice: 99.99'

In [30]:
agent_chain.run(input="Yes also show me a picture of it")

'\nhttps://burst.shopifycdn.com/photos/cafe-patio_925x.jpg'